In [1]:
import numpy as np

docWord = np.load("science2k-doc-word.npy")
docWord[0]

array([-0.2521619, -0.2521619,  9.36371  , ..., -0.2521619, -0.2521619,
       -0.2521619])

### k-means
1. Select k points as means
2. Assign each point to its closest center
3. Compute clusters centers
4. Repeat 2-4

In [57]:
def assignPointsToClusters(points, clustersCenters, k):
    assignment = np.empty(points.shape[0])
    withinDist = 0
    for i in range(points.shape[0]):
        minDist = np.infty
        for cluster in range(k):
            dist = np.linalg.norm(points[i] - clustersCenters[cluster])
            if dist < minDist:
                minDist = dist
                bestCluster = cluster
        assignment[i] = bestCluster
        withinDist += minDist
    return assignment, withinDist

def computeClustersCenters(points, assignment, k):
    clustersCenters = np.empty((k,points.shape[1]))
    for cluster in range(k):
        center = np.mean(points[assignment == cluster], axis=0)
        clustersCenters[cluster] = center
    return clustersCenters

def kMeans(points, k, n_iter):
    withinDists = np.empty(n_iter)
    for i in range(n_iter):
        clustersCenters = points[np.random.randint(0,points.shape[0],k)]
        withinDist = 0
        oldWithinDist = -1
        while withinDist != oldWithinDist:
            oldWithinDist = withinDist
            assignment, withinDist = assignPointsToClusters(points, clustersCenters, k)
            clustersCenters = computeClustersCenters(points, assignment, k)
            print('W'+str(i)+' = ' + str(round(withinDist,5)), end='\r')
        withinDists[i] = withinDist
    minWithinDist = np.min(withinDists)
    stdWithinDist = np.std(withinDists)
    return minWithinDist, stdWithinDist

for k in range(2,21):
    print('k = '+str(k))
    minWithinDist, std = kMeans(docWord, k, n_iter=6)
    print('W = '+str(minWithinDist))
    print('CI=['+str(minWithinDist-std)+' - '+str(minWithinDist+std)+']')


k = 2
W = 211655.1753113372
CI=[210452.84950721133 - 212857.50111546306]
k = 3
W = 209324.6013378002
CI=[207842.78962273602 - 210806.4130528644]
k = 4
W = 208535.2205599403
CI=[206698.65228015327 - 210371.78883972732]
k = 5
W = 207974.41487964336
CI=[206531.3696899517 - 209417.460069335]
k = 6
W = 207466.20832556474
CI=[207165.63776626432 - 207766.77888486517]
k = 7
W = 207587.2160021817
CI=[206292.79886901355 - 208881.63313534984]
k = 8
W = 207163.38404277898
CI=[205645.70207946617 - 208681.0660060918]
k = 9
W = 206343.9816190579
CI=[205537.91354912415 - 207150.04968899162]
k = 10
W = 206859.85472839294
CI=[206387.36262736024 - 207332.34682942563]
k = 11
W = 206119.70612083227
CI=[205707.28186228592 - 206532.13037937862]
k = 12


C:\Users\Romain\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Romain\Anaconda3\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


W = 205949.90536411482
CI=[205523.9559328682 - 206375.85479536143]
k = 13
W = 205828.7788135948
CI=[205276.42318169322 - 206381.1344454964]
k = 14
W = 205709.1359562965
CI=[205090.375041927 - 206327.89687066598]
k = 15
W = 205027.27753835928
CI=[204498.77087514743 - 205555.78420157114]
k = 16
W = 205309.0687536697
CI=[204916.38632020965 - 205701.75118712976]
k = 17
W = 204740.12403670157
CI=[204570.51567327115 - 204909.732400132]
k = 18
W = 204856.49028692645
CI=[204346.04552449353 - 205366.93504935937]
k = 19
W = 204167.21905232672
CI=[203602.30525217063 - 204732.1328524828]
k = 20
W = 204119.86052397487
CI=[203552.36692403673 - 204687.354123913]


We will choose k=9 moving forward, because it achieved a reasonable within-cluster distance compared to higher k's while being probably easier to interpret (smaller number of clusters).

In [49]:
kMeans(docWord, 10, n_iter=6)

207271.371390612